<a href="https://colab.research.google.com/github/nurshatfateh/WebScraping-NaturalLanguageProcessing/blob/main/trial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [280]:
import requests
import json
import pandas
import spacy
from spacy.matcher import Matcher

In [281]:
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [282]:
import newspaper
from newspaper import Article

In [283]:
nlp = spacy.load('en_core_web_sm')

In [284]:
def extract_article_from_url(url):
  a = Article(url)
  a.download()
  a.parse()
  return a.text, a.publish_date


In [285]:
def sentence_giver(doc, start, end):
  s = start
  while str(doc[s]) != '.':
    s = s - 1
    if s < 0:
      break
  e = end
  while str(doc[e]) != '.':
    e = e + 1
    if e > len(doc) - 1:
      break
  return str(doc[s+1:e])

In [286]:
def location_finder(doc):
  locations = []
  sentences = []
  matcher = Matcher(nlp.vocab)
  pattern = [{"ENT_TYPE": "GPE"}]
  matcher.add("Location", [pattern])
  matches = matcher(doc)
  for match_id, start, end in matches:
    sentence = sentence_giver(doc, start, end)
    value = l_detail_filter(sentence)
    if value == 1:
      sentences.append(sentence)
      locations.append(doc[start:end])
  return locations, sentences

def l_detail_filter(data):
  data = nlp(data)
  matcher = Matcher(nlp.vocab)
  pattern1 = [{"LEMMA": "accident", "POS": "VERB"},{"LOWER": "in"}]
  pattern2 = [{"LEMMA": "die", "POS": "VERB"},{"LOWER": "in"}]
  pattern3 = [{"LEMMA": "collide", "POS": "VERB"},{"LOWER": "in"}]
  pattern4 = [{"LEMMA": "crash", "POS": "VERB"},{"LOWER": "in"}]
  pattern5 = [{"LEMMA": "kill", "POS": "VERB"},{"LOWER": "in"}]
  pattern0 = [{"LOWER": "in"}]

  matcher.add("N_Filter", [pattern1])
  matches = matcher(data)
  if len(matches) != 0:
    return 1
  else:
    matcher.add("N_Filter", [pattern2])
    matches = matcher(data)
    if len(matches) != 0:
      return 1
    else:
      matcher.add("N_Filter", [pattern3])
      matches = matcher(data)
      if len(matches) != 0:
        return 1
      else:
        matcher.add("N_Filter", [pattern4])
        matches = matcher(data)
        if len(matches) != 0:
          return 1
        else:
          matcher.add("N_Filter", [pattern5])
          matches = matcher(data)
          if len(matches) != 0:
            return 1
          else:
            matcher.add("N_Filter", [pattern0])
            matches = matcher(data)
            if len(matches) != 0:
              return 1
            else:
              return 0

In [287]:
def number_finder(doc):
  numbers = []
  sentences = []
  matcher = Matcher(nlp.vocab)
  pattern = [{"LIKE_NUM": True}]
  matcher.add("Number", [pattern])
  matches = matcher(doc)
  for match_id, start, end in matches:
    sentence = sentence_giver(doc, start, end)
    value = n_detail_filter(sentence)
    if value == 1:
      sentences.append(sentence)
      numbers.append(str(doc[start:end]))
  return numbers, sentences

def n_detail_filter(data):
  data = nlp(data)
  matcher = Matcher(nlp.vocab)
  pattern1 = [{"LEMMA": "kill", "POS": "VERB"}]
  pattern2 = [{"LEMMA": "die", "POS": "VERB"}]


  matcher.add("N_Filter", [pattern1])
  matches = matcher(data)
  if len(matches) != 0:
    return 1
  else:
    matcher.add("N_Filter", [pattern2])
    matches = matcher(data)
    if len(matches) != 0:
      return 1
    else:
      return 0
        
    

In [288]:
link = 'https://www.ndtv.com/india-news/2-cops-killed-as-speeding-truck-hits-bike-in-bihar-3472041'
content, date = extract_article_from_url(link)
doc = nlp(content)


l_places, l_s = location_finder(doc)
print("ACCIDENT LOCATION: ",l_places)
print(l_s)

n_people, n_s = number_finder(doc)
if len(n_s)==0:
  print("DEATH NUMBER: ",1)
else:
  print("DEATH NUMBER: ",n_people[0])
print(n_s)

print("RELATED VEHICLES: ")
vehicle = ['truck','bus','car','bike','cycle','rickshaw','tempo']
for i in vehicle:
  for j in doc.text.split():
    if i in j:
      if j!='carrying':
        print(j)
        break
      






  



ACCIDENT LOCATION:  [Bihar, Bhojpur]
["(Representational)\n\nTwo policemen were killed today after the motorcycle they were riding was knocked down by a truck in Bihar's Bhojpur district, an officer said", "(Representational)\n\nTwo policemen were killed today after the motorcycle they were riding was knocked down by a truck in Bihar's Bhojpur district, an officer said"]
DEATH NUMBER:  Two
["(Representational)\n\nTwo policemen were killed today after the motorcycle they were riding was knocked down by a truck in Bihar's Bhojpur district, an officer said"]
RELATED VEHICLES: 
truck
motorcycle


In [289]:
link = 'https://www.ndtv.com/india-news/cctv-shows-truck-losing-control-on-road-in-up-3-dead-as-it-rams-auto-3463172'
content, date = extract_article_from_url(link)
doc = nlp(content)


l_places, l_s = location_finder(doc)
print("ACCIDENT LOCATION: ",l_places)
print(l_s)

n_people, n_s = number_finder(doc)
if len(n_s)==0:
  print("DEATH NUMBER: ",1)
else:
  print("DEATH NUMBER: ",n_people[0])
print(n_s)

print("RELATED VEHICLES: ")
vehicle = ['truck','bus','car','bike','cycle','rickshaw','tempo']
for i in vehicle:
  for j in doc.text.split():
    if i in j:
      if j!='carrying':
        print(j)
        break

ACCIDENT LOCATION:  [Bardah]
["\n\nAccording to the police, the accident occurred near Thekma in Azamgarh's Bardah around 10 pm on Tuesday"]
DEATH NUMBER:  Three
["\n\nThree people died and four others were injured after a loader truck collided with an autorickshaw in Uttar Pradesh's Azamgarh on Tuesday night", "\n\nThree people died and four others were injured after a loader truck collided with an autorickshaw in Uttar Pradesh's Azamgarh on Tuesday night", 'A 17-year-old girl died on the spot while those injured were rushed to a nearby hospital where later, a child and an elderly man also died of injuries," said a police official']
RELATED VEHICLES: 
truck
autorickshaw


In [290]:
link = 'https://www.ndtv.com/india-news/4-dead-after-2-motorcycles-collide-in-uttar-pradesh-police-3460331'
content, date = extract_article_from_url(link)
doc = nlp(content)


l_places, l_s = location_finder(doc)
print("ACCIDENT LOCATION: ",l_places)
print(l_s)

n_people, n_s = number_finder(doc)
if len(n_s)==0:
  print("DEATH NUMBER: ",1)
else:
  print("DEATH NUMBER: ",n_people[0])
print(n_s)

print("RELATED VEHICLES: ")
vehicle = ['truck','bus','car','bike','cycle','rickshaw','tempo']
for i in vehicle:
  for j in doc.text.split():
    if i in j:
      if j!='carrying':
        print(j)
        break


      

ACCIDENT LOCATION:  [Shyamdeura]
['\n\nFour persons were killed while another was injured when two motorcycles collided on the Maharajganj-Gorakhpur road in Shyamdeura area, police said on Tuesday']
DEATH NUMBER:  Four
['\n\nFour persons were killed while another was injured when two motorcycles collided on the Maharajganj-Gorakhpur road in Shyamdeura area, police said on Tuesday', '\n\nFour persons were killed while another was injured when two motorcycles collided on the Maharajganj-Gorakhpur road in Shyamdeura area, police said on Tuesday']
RELATED VEHICLES: 
bike
motorcycles


In [291]:
link = 'https://www.ndtv.com/mumbai-news/boy-12-killed-in-mumbai-after-being-run-over-by-bus-driver-arrested-3453328'
content, date = extract_article_from_url(link)
doc = nlp(content)


l_places, l_s = location_finder(doc)
print("ACCIDENT LOCATION: ",l_places)
print(l_s)

n_people, n_s = number_finder(doc)
if len(n_s)==0:
  print("DEATH NUMBER: ",1)
else:
  print("DEATH NUMBER: ",n_people[0])
print(n_s)

print("RELATED VEHICLES: ")
vehicle = ['truck','bus','car','bike','cycle','rickshaw','tempo']
for i in vehicle:
  for j in doc.text.split():
    if i in j:
      if j!='carrying':
        print(j)
        break

ACCIDENT LOCATION:  [Shivajinagar]
['\n\nThe accident took place near Nurani Masjid in Shivajinagar area on Thursday evening, an official said']
DEATH NUMBER:  1
[]
RELATED VEHICLES: 
bus


In [279]:
link = 'https://www.ndtv.com/india-news/4-killed-after-bus-carrying-pilgrims-overturns-in-jharkhand-police-3396586'
content, date = extract_article_from_url(link)
doc = nlp(content)

l_places, l_s = location_finder(doc)
print("ACCIDENT LOCATION: ",l_places)
print(l_s)

n_people, n_s = number_finder(doc)
if len(n_s)==0:
  print("DEATH NUMBER: ",1)
else:
  print("DEATH NUMBER: ",n_people[0])
print(n_s)

print("RELATED VEHICLES: ")
vehicle = ['truck','bus','car','bike','cycle','rickshaw','tempo']
for i in vehicle:
  for j in doc.text.split():
    if i in j:
      if j!='carrying':
        print(j)
        break
      

ACCIDENT LOCATION:  [Hazaribag, district]
['\n\nFour people were killed and 25 others injured when the bus they were travelling in overturned in Hazaribag district, a senior police officer said', '\n\nFour people were killed and 25 others injured when the bus they were travelling in overturned in Hazaribag district, a senior police officer said']
DEATH NUMBER:  Four
['\n\nFour people were killed and 25 others injured when the bus they were travelling in overturned in Hazaribag district, a senior police officer said', '\n\nFour people were killed and 25 others injured when the bus they were travelling in overturned in Hazaribag district, a senior police officer said']
RELATED VEHICLES: 
bus
